In [458]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import numpy as np

In [233]:
main_url = "https://www.google.com/about/careers/applications/jobs/results/"
response = requests.get(main_url)
print(response)

<Response [200]>


In [194]:
doc = BeautifulSoup(response.text, "html.parser")

In [195]:
job_listings = int(
    doc.find(class_='VfPpkd-wZVHld-gruSEe-j4LONd').text[-4:]
)
print(f'Total Job Listings: {job_listings}')

total_pages = job_listings//20 + 1
print(f'Total Pages: {total_pages}')

Total Job Listings: 2553
Total Pages: 128


In [196]:
rel_urls = []

for _ in tqdm(range(1,total_pages+1)):
    page_url = f"https://www.google.com/about/careers/applications/jobs/results/?page={_}"
    response = requests.get(page_url)
    doc = BeautifulSoup(response.text, "html.parser")
    jobs = doc.find_all(class_='lLd3Je')
    for job in jobs:
        rel_urls.append(
            job.find(class_='WpHeLc VfPpkd-mRLv6 VfPpkd-RLmnJb')['href']
        )

print(len(rel_urls))

100%|████████████████████████████████████████████████████████████████████████████████| 128/128 [06:29<00:00,  3.04s/it]

2553


In [429]:
rel_urls_s = pd.Series(rel_urls).str[13:]
rel_urls_s

0       84039463755227846-program-manager-data-center-...
1       125615186340913862-api-management-engineer?page=1
2       132800609181934278-customer-engineer-ai-google...
3       106629691430838982-product-manager-forms-of-pa...
4       117104680189731526-head-of-games-and-scale-str...
                              ...                        
2548    108357684817732294-software-developer-iii-site...
2549    85317367386514118-senior-software-engineer-chr...
2550    102045983952511686-senior-software-engineer-ha...
2551    75355036929925830-senior-staff-software-engine...
2552    91135340554658502-software-engineer-iii-engine...
Length: 2553, dtype: object

In [ ]:
# scraping job data and collecting in dataframe
for _ in tqdm(range(98,200),desc='Scraping and Creating a Pandas DataFrame',colour='#A020F0',ascii=True):

    job_url = main_url + rel_urls_s[_]
    response = requests.get(job_url)
    doc = BeautifulSoup(response.text,'html.parser')
    
    # lists for temp dataframe
    org = []; jbt = []; loc = []; exp = []; rmt = [];
    mnq = []; pfq = []; jbd = []; rsp = []; url = [];
    
    # dictionary for temp dataframe
    dct = {
        'Job Title':jbt,
        'Organisation':org,
        'Location':loc,
        'Experience':exp,
        'Remote Work':rmt,
        'Minimum Qualification':mnq,
        'Preferred Qualification':pfq,
        'Job Description':jbd,
        'Responsibilities':rsp,
        'URL':url
    }

    # job title
    jbt.append(doc.find(class_='p1N2lc').text)
    # organisation
    org.append(doc.find(class_='RP7SMd').span.text)
    # location
    loc.append(doc.find(class_='r0wTof').text)
    # experience
    try:
        exp.append(doc.find(class_='wVSTAb').text)
    except AttributeError as e:
        exp.append(doc.find_all(class_='RP7SMd')[-1].span.text)
    # remote eligibility
    if len(doc.find_all(class_='RP7SMd')) != 1:
        if doc.find_all(class_='RP7SMd')[1].span.text.lower() == 'remote eligible':
            rmt.append(True)
        else:
            rmt.append(False)
    else:
        rmt.append(False)
    # minimum qualification
    try:
        mnq.append(doc.find(class_='KwJkGe').find_all('ul')[0].text.replace('\n',''))
    except IndexError:
        mnq.append(np.nan)
        pass
    except AttributeError:
        mnq.append(np.nan)
        pass
    # preferred qualification
    try:
        pfq.append(doc.find(class_='KwJkGe').find_all('ul')[1].text.replace('\n',''))
    except IndexError:
        pfq.append(np.nan)
        pass
    except AttributeError:
        pfq.append(np.nan)
        pass
    # job description
    jbd.append(doc.find(class_='aG5W3').text.replace('\n','')[13:])
    # responsilibities
    try:
        rsp.append(doc.find(class_='BDNOWe').text[16:].replace('\n',''))
    except AttributeError:
        rsp.append(np.nan)
        pass
    # job url
    url.append(job_url)
    
    # creating a temp dataframe
    col_names = ['Job Title','Organisation','Location','Experience','Remote Work',
             'Minimum Qualification','Preferred Qualification',
             'Job Description','Responsibilities','URL']
    
    temp = pd.DataFrame(data=dct,columns=col_names)
    
    # appending entries into dataframe
    google_jobs_df = pd.concat([google_jobs_df,temp])
    google_jobs_df = google_jobs_df.reset_index().drop(columns='index')

Scraping and Creating a Pandas DataFrame:  24%|#########1                             | 24/102 [01:01<03:37,  2.79s/it]

In [644]:
google_jobs_df

,Job Title,Organisation,Location,Experience,Remote Work,Minimum Qualification,Preferred Qualification,Job Description,Responsibilities,URL
0,"Program Manager, Data Center Environmental, He...",Google,"London, UK",Mid,NaN,Bachelor's degree in Occupational Health and S...,Credentials from Board of Certified Safety Pro...,A problem isn’t truly solved until it’s solved...,Lead the development and ownership of the EHS ...,https://www.google.com/about/careers/applicati...
1,API Management Engineer,Google,"London, UK",Mid,NaN,"Bachelor's degree in Computer Science, Mathema...",Experience in implementing API Security and Th...,The Google Cloud Platform team helps customers...,Work closely with customers on their API progr...,https://www.google.com/about/careers/applicati...
2,"Customer Engineer, AI, Google Cloud",Google,"London, UK",Mid,NaN,Bachelor's degree or equivalent practical expe...,"Master's degree in Computer Science, or a rela...","When leading companies choose Google Cloud, it...","Build partner specific, go-to-market solutions...",https://www.google.com/about/careers/applicati...
3,"Product Manager, Forms of Payment Platform, Go...",Google,"London, UK",Mid,NaN,Bachelor's degree or equivalent practical expe...,Master's degree or PhD in a Technical field.Ex...,"At Google, we put our users first. The world i...",Partner closely with engineering to define fut...,https://www.google.com/about/careers/applicati...
4,"Head of Games and Scale, Strategy and Operations",Google,"London, UK",Advanced,NaN,Bachelor's degree or equivalent practical expe...,Experience operating within and leading cross-...,The Play Partnerships team works with the deve...,"Manage, lead, and develop a team to drive glob...",https://www.google.com/about/careers/applicati...
...,...,...,...,...,...,...,...,...,...,...
99,"Site Lead, Data Center Operations",Google,"Fremont, CA, USA",Advanced,False,Bachelor's degree in a technical field or equi...,Experience working in data center environments...,Google isn't just a software company. The Hard...,"Lead a team of individuals, set and communicat...",https://www.google.com/about/careers/applicati...
100,"Critical Incident Manager, Cloud",Google,"Fremont, CA, USA",Mid,False,Bachelor's degree or equivalent practical expe...,Certification in program management.Experience...,A problem isn’t truly solved until it’s solved...,Act as the primary point of contact for key st...,https://www.google.com/about/careers/applicati...
101,"Chief of Staff, YouTube Business",YouTube,"Fremont, CA, USA",Mid,False,Bachelor's degree or equivalent practical expe...,"MBA or Master's degree in a technical, quantit...",As the Chief of Staff Team for YouTube’s Chief...,Set strategic priorities for the YouTube Busin...,https://www.google.com/about/careers/applicati...
102,"Open Career Opportunities, Verily Life Sciences",Verily Life Sciences,"Fremont, CA, USA",Verily Life Sciences,False,NaN,NaN,"Verily, an Alphabet company, lives at the inte...",NaN,https://www.google.com/about/careers/applicati...


In [685]:
jbt,org,loc,exp,rmt,mnq,pfq,jbd,rsp,url

(['Legal Operations Specialist, YouTube'],
 ['YouTube'],
 ['Fremont, CA, USA'],
 ['Early'],
 [False],
 ["Bachelor's degree or equivalent practical experience.1 year of experience working in law, policy, legal regulation, project management, or operations.",
  nan],
 ['2 years of experience in content moderation, law, policy, operations, or a related field.Experience working in internet technologies and related policy, and regulatory environments.Ability to communicate in English and another local Asian language (e.g Hindi, Mandarin, Cantonese, Vietnamese, Bahasa Indonesia, Malay, or Japanese) fluently to work with key regional partners across APAC.Excellent communication, people management, presentation, and project management skills.',
  nan],
 ["Our Legal team needs more than attorneys to keep things moving. Our Legal Operations team is charged with improving and maintaining the Legal team's internal processes, systems and tools. Responsible for tracking objectives, coordinating fina

In [693]:
job_url

'https://www.google.com/about/careers/applications/jobs/results/124491726204084934-open-engineering-career-opportunities-capitalg-portfolio-companies?page=5'

In [253]:
job_url = main_url + rel_urls_s[21]
job_url

'https://www.google.com/about/careers/applications/jobs/results/140130980726743750-senior-director-gsrs-resilience-and-risk-foresight?page=2'

In [255]:
response = requests.get(job_url)
print(response)
doc = BeautifulSoup(response.text,'html.parser')

<Response [200]>


In [408]:
job_title = doc.find(class_='p1N2lc').text
job_title

'Overnight Principal Incident Response Consultant, Mandiant, Google Cloud'

In [410]:
org_name = doc.find(class_='RP7SMd').span.text
org_name

'Google'

In [412]:
location = doc.find(class_='r0wTof').text
location

'London, UK'

```
experience = doc.find(class_='wVSTAb').text  
experience
```

In [415]:
try:
    experience = doc.find(class_='wVSTAb').text
except AttributeError as e:
    experience = doc.find_all(class_='RP7SMd')[-1].span.text
    
experience

'Advanced'

In [417]:
min_qual = doc.find(class_='KwJkGe').find_all('ul')[0].text.replace('\n','')
min_qual

"Bachelor's degree in Computer Science, Information Systems, Cybersecurity, a related technical field, or equivalent practical experience.6 years of experience working with incident response investigations, analysis, and containment actions.6 years of experience with network forensics, malware triage analysis, cloud forensics, and disk and memory forensics.Ability to work non-standard hours and differing rotations/shifts."

In [419]:
pref_qual= doc.find(class_='KwJkGe').find_all('ul')[1].text.replace('\n','')
pref_qual

'Certification in Google Cloud Platforms (GCP).Experience with Windows, MacOS, Linux operating system and memory forensics.Experience working in fast-moving and changing environments, with the ability to present written or verbal information to technical and non-technical audiences.Knowledge of tools used in pen testing, security event analysis, incident response, computer forensics, network and endpoint architecture, or other security operational areas.Ability to set and manage expectations with executive stakeholders and team members and prioritize, execute and pivot to other tasks with little warning.Excellent problem-solving, troubleshooting, and analysis skills.'

In [421]:
about_job = doc.find(class_='aG5W3').text.replace('\n','')[13:]
about_job

'As a Security Consultant, you will be responsible for helping clients effectively prepare for, proactively mitigate, and detect and respond to cyber security threats. Security Consultants have an understanding of computer science, operating system functionality and networking, cloud services, corporate network environments and how to apply this knowledge to cyber security threats.  As a Security Consultant, you could work on engagements including assisting clients in navigating technically complex and high-profile incidents, performing forensic analysis, threat hunting, and malware triage. You may also test client networks, applications and devices by emulating the latest techniques to help them defend against threats, and will be the technical advocate for information security requirements and provide an in-depth understanding of the information security domain. You will also articulate and present complex concepts to business stakeholders, executive leadership, and technical contrib

In [427]:
resp = doc.find(class_='BDNOWe').text[16:].replace('\n','')
resp

'Lead\xa0large, client-facing incident response engagements, examine cloud, endpoint, and network-based sources of evidence.Recognize and codify attacker Tools, Tactics, and Procedures (TTPs) and Indicators of Compromise (IOCs) that can be applied to current and future investigations.Build scripts, tools, or methodologies to enhance Mandiant’s incident investigation processes.Develop and present comprehensive and accurate reports, training, and presentations for both technical and executive audiences.Conduct host forensics, network forensics, log analysis, and malware triage in support of incident response investigations.'

In [404]:
tmp_url = 'https://www.google.com/about/careers/applications/jobs/results/113883636985406150-overnight-principal-incident-response-consultant-mandiant-google-cloud'
result = requests.get(tmp_url).text
doc = BeautifulSoup(result,'html.parser')

In [405]:
if doc.find_all(class_='RP7SMd')[1].span.text.lower() == 'remote eligible':
    remote = True
else:
    remote = False

remote

True

In [400]:
doc.find_all(class_='RP7SMd')

[<span class="RP7SMd"><i aria-hidden="true" class="google-material-icons VfPpkd-kBDsod dPX0he">corporate_fare</i><span>Google</span></span>,
 <span class="RP7SMd"><i aria-hidden="true" class="google-material-icons VfPpkd-kBDsod dPX0he">laptop_windows</i><span>Remote eligible</span></span>,
 <span class="RP7SMd"><i aria-hidden="true" class="google-material-icons VfPpkd-kBDsod dPX0he">bar_chart</i><span>Director+</span></span>]

In [437]:
import time

for i in tqdm(range(15),desc='stuff',colour='#A020F0',ascii=True):
    time.sleep(0.1)

stuff: 100%|###########################################################################| 15/15 [00:01<00:00,  9.50it/s]


In [457]:

dct = {1:gob,5:bob}
gob = [6,3]
bob = [3.2,0.1]
sample = pd.DataFrame(data=dct,columns=[1,5])
sample

NameError: name 'gob' is not defined

In [423]:
ls1 = [2,6]
pd.

AttributeError: 'DataFrame' object has no attribute 'vstack'

In [531]:
col_names = ['Job Title','Organisation','Location','Experience',
             'Minimum Qualification','Preferred Qualification',
             'Job Description','Responsibilities','URL']

google_jobs_df = pd.DataFrame(columns=col_names)
google_jobs_df

,Job Title,Organisation,Location,Experience,Minimum Qualification,Preferred Qualification,Job Description,Responsibilities,URL


In [541]:
for _ in tqdm(range(10),desc='stuff',colour='green',ascii=True):
    job_url = main_url + rel_urls_s[_]
    response = requests.get(job_url)
    doc = BeautifulSoup(response.text,'html.parser')
    
    org = []; jbt = []; loc = []; exp = []; mnq = [];
    pfq = []; jbd = []; rsp = []; url = [];
    
    dct = {
        'Job Title':jbt,
        'Organisation':org,
        'Location':loc,
        'Experience':exp,
        'Minimum Qualification':mnq,
        'Preferred Qualification':pfq,
        'Job Description':jbd,
        'Responsibilities':rsp,
        'URL':url
    }
    # job title
    jbt.append(doc.find(class_='p1N2lc').text)
    # organisation
    org.append(doc.find(class_='RP7SMd').span.text)
    # location
    loc.append(doc.find(class_='r0wTof').text)
    # experience
    exp.append(doc.find(class_='wVSTAb').text)
    # minimum qualification
    mnq.append(doc.find(class_='KwJkGe').find_all('ul')[0].text.replace('\n',''))
    # preferred qualification
    pfq.append(doc.find(class_='KwJkGe').find_all('ul')[1].text.replace('\n',''))
    # job description
    jbd.append(doc.find(class_='aG5W3').text[17:].replace('\n',''))
    # responsilibities
    rsp.append(doc.find(class_='BDNOWe').text[16:].replace('\n',''))
    # job url
    url.append(job_url)
    
    col_names = ['Job Title','Organisation','Location','Experience',
             'Minimum Qualification','Preferred Qualification',
             'Job Description','Responsibilities','URL']
    temp = pd.DataFrame(data=dct,columns=col_names)
    
    google_jobs_df = pd.concat([google_jobs_df,temp])
    google_jobs_df = google_jobs_df.reset_index().drop(columns='index')

stuff: 100%|###########################################################################| 10/10 [00:30<00:00,  3.04s/it]


In [571]:
print("\U0001F389"*5)

🎉🎉🎉🎉🎉


In [567]:
print("\U0001F603")

😃


In [579]:
print('Mission Complete '+'\U0001F389'*5)

Mission Complete 🎉🎉🎉🎉🎉


In [431]:
col_names = ['Job Title','Organisation','Location','Experience',
             'Remote Work','Minimum Qualification',
             'Preferred Qualification','Job Description',
             'Responsibilities','URL']

google_jobs_df = pd.DataFrame(columns=col_names)